# Data Processing Notebook
Notebook for the cleaning and merging of dataframes from csv files for the intention of loading to one csv

In [1]:
#imports
import pandas as pd

## Dataset Loading

In [19]:
#Load in all datasets
edu = pd.read_csv("../data/raw/Education.csv", encoding='latin-1')
conn = pd.read_csv("../data/raw/county_broadband_adoption.csv", encoding='utf-8')
pov = pd.read_csv("../data/raw/poverty_estimates.csv", encoding='latin-1')
unem = pd.read_csv("../data/raw/unemployment.csv", encoding='latin-1', thousands=',')
urban = pd.read_csv("../data/raw/urban_rural_classification.csv", encoding='utf-8')

## Dataset Cleaning

### Education Dataset
The main dataset we want for classification, labeling, and response. Mainly cleaned out less useful metrics and used percentages from years 2014-2018 as that's the more relevant time period.

In [6]:
edu_cleaned = edu.rename(columns={
    "FIPS Code": "fips",
    "Percent of adults with less than a high school diploma, 2014-18": "perc_less_highschool",
    "Percent of adults with a high school diploma only, 2014-18": "perc_highschool",
    "Percent of adults completing some college or associate's degree, 2014-18": "perc_some_college",
    "Percent of adults with a bachelor's degree or higher, 2014-18": "perc_college"
})
edu_cleaned = edu_cleaned.set_index('fips')
edu_cleaned = edu_cleaned.drop([0])
edu_cleaned = edu_cleaned[["perc_less_highschool", "perc_highschool", "perc_some_college", "perc_college"]]
edu_cleaned.head()

,perc_less_highschool,perc_highschool,perc_some_college,perc_college
fips,,,,
1000,14.2,30.9,29.9,24.9
1001,11.3,32.6,28.4,27.7
1003,9.7,27.6,31.3,31.3
1005,27.0,35.7,25.1,12.2
1007,16.8,47.3,24.4,11.5


### Internet Connectivity Dataset
This gives broadband adoption data as a percentage over the past 2 decades. We're going to utilize an average of the span of 2014-2018 since the other datasets are formatted in such a manner.

In [8]:
#Index Setting
conn_cleaned = conn.rename(columns={"cfips": "fips"})
conn_cleaned = conn_cleaned.set_index("fips")

In [10]:
conn_cleaned = conn_cleaned[conn_cleaned["year"] >= 2014]
conn_cleaned = conn_cleaned[conn_cleaned["year"] <= 2018]
conn_cleaned = conn_cleaned.groupby(["fips"]).mean()
conn_cleaned = conn_cleaned[["broadband"]]
conn_cleaned.head()

,broadband
fips,
1001,0.703591
1003,0.771106
1005,0.541496
1007,0.633871
1009,0.635847


### Unemployment and Income
A confounding variable we'd want to check for. We take the unemployment and median house income from 2018 by county fips code

In [11]:
unem_cleaned = unem.rename(columns={
    "FIPStxt": "fips",
    "Unemployment_rate_2019": "unemployment_rate",
    "Median_Household_Income_2018": "median_house_income"
})
unem_cleaned = unem_cleaned.set_index("fips")
unem_cleaned = unem_cleaned.drop([0])
unem_cleaned = unem_cleaned[["unemployment_rate", "median_house_income"]]
unem_cleaned.head()

,unemployment_rate,median_house_income
fips,,
1000,3.0,49881.0
1001,2.7,59338.0
1003,2.7,57588.0
1005,3.8,34382.0
1007,3.1,46064.0


### Urbanization
We also want to add urbanization indexes. This is a 1 to 6 classification where is a large central metro and 6 is a non-core rural area.

In [16]:
urban_cleaned = urban.rename(columns = {
    "2013 code": "urbanization_class",
    "FIPS code": "fips"
})
urban_cleaned = urban_cleaned.set_index("fips")
urban_cleaned = urban_cleaned[["urbanization_class"]]
urban_cleaned.head()

,urbanization_class
fips,
1001,3
1003,4
1005,6
1007,2
1009,2


### Poverty
The estimated percentages of people living in poverty could also prove useful.

In [26]:
pov_cleaned = pov.rename(columns={
    "FIPStxt": "fips",
    "PCTPOVALL_2018": "poverty_percentage",
    "PCTPOV017_2018": "poverty_percentage_0-17"
})
pov_cleaned = pov_cleaned.set_index("fips")
pov_cleaned = pov_cleaned.drop([0])
pov_cleaned = pov_cleaned[["poverty_percentage", "poverty_percentage_0-17"]]
pov_cleaned.head()

,poverty_percentage,poverty_percentage_0-17
fips,,
1000,16.8,23.9
1001,13.8,19.3
1003,9.8,13.9
1005,30.9,43.9
1007,21.8,27.8


## Dataset Merging

In [28]:
df = conn_cleaned.join(edu_cleaned, on="fips")
df = df.join(unem_cleaned, on="fips")
df = df.join(urban_cleaned, on="fips")
df = df.join(pov_cleaned, on="fips")
df.head()

,broadband,perc_less_highschool,perc_highschool,perc_some_college,perc_college,unemployment_rate,median_house_income,urbanization_class,poverty_percentage,poverty_percentage_0-17
fips,,,,,,,,,,
1001,0.703591,11.3,32.6,28.4,27.7,2.7,59338.0,3.0,13.8,19.3
1003,0.771106,9.7,27.6,31.3,31.3,2.7,57588.0,4.0,9.8,13.9
1005,0.541496,27.0,35.7,25.1,12.2,3.8,34382.0,6.0,30.9,43.9
1007,0.633871,16.8,47.3,24.4,11.5,3.1,46064.0,2.0,21.8,27.8
1009,0.635847,19.8,34.0,33.5,12.6,2.7,50412.0,2.0,13.2,18.0


## Write to CSV

In [29]:
df.to_csv("../data/dataframe.csv", index=True)

## FIPS
I also want an association list of FIPS Code to state and area

In [35]:
fips = edu.rename(columns={
    "FIPS Code": "fips",
    "State": "state",
    "Area name": "area"
})
fips = fips.drop([0])
fips = fips.set_index('fips')
fips = fips[["state", "area"]]
fips.head()

,state,area
fips,,
1000,AL,Alabama
1001,AL,Autauga County
1003,AL,Baldwin County
1005,AL,Barbour County
1007,AL,Bibb County


In [36]:
fips.to_csv("../data/fips.csv", index=True)